In [1]:
from flask import Flask
from flask import request, jsonify
import json
import datetime
import hashlib
import time
#from pysimplechain import Block, SimpleChain, Message

In [2]:
class Message:
	def __init__(self, data):
		self.hash = None
		self.prev_hash = None
		self.timestamp = time.time()
		self.size = len(data.encode('utf-8'))   # length in bytes
		self.data = data
		self.payload_hash = self._hash_payload()

	def _hash_payload(self):
		return hashlib.sha256(bytearray(str(self.timestamp) + str(self.data), "utf-8")).hexdigest()

	def _hash_message(self):
		return hashlib.sha256(bytearray(str(self.prev_hash) + self.payload_hash, "utf-8")).hexdigest()

	def link(self, message):
		""" Link the message to the previous one via hashes."""
		self.prev_hash = message.hash

	def seal(self):
		""" Get the message hash. """
		self.hash = self._hash_message()

	def validate(self):
		""" Check whether the message is valid or not. """
		if self.payload_hash != self._hash_payload():
			raise InvalidMessage("Invalid payload hash in message: " + str(self))
		if self.hash != self._hash_message():
			raise InvalidMessage("Invalid message hash in message: " + str(self))

	def __repr__(self):
		return 'Message<hash: {}, prev_hash: {}, data: {}>'.format(
			self.hash, self.prev_hash, self.data[:20]
		)


class Block:
	def __init__(self, *args):
		self.messages = []
		self.timestamp = None
		self.prev_hash = None
		self.hash = None
		if args:
			for arg in args:
				self.add_message(arg)

	def _hash_block(self):
		return hashlib.sha256(bytearray(str(self.prev_hash) + str(self.timestamp) + self.messages[-1].hash, "utf-8")).hexdigest()

	def add_message(self, message):
		if len(self.messages) > 0:
			message.link(self.messages[-1])
		message.seal()
		message.validate()
		self.messages.append(message)
 
	def link(self, block):
		""" The block hash only incorporates the head message hash
			which then transitively includes all prior hashes.
		"""
		self.prev_hash = block.hash
        
	def seal(self):
		self.timestamp = time.time()
		self.hash = self._hash_block()

	def validate(self):
		""" Validates each message hash, then chain integrity, then the block hash.
			Calls each message's validate() method.

			If a message fails validation, this method captures the exception and 
			throws InvalidBlock since an invalid message invalidates the whole block.
		"""
		for i, msg in enumerate(self.messages):
			try:
				msg.validate()
				if i > 0 and msg.prev_hash != self.messages[i-1].hash:
					raise InvalidBlock("Invalid block: Message #{} has invalid message link in block: {}".format(i, str(self)))
			except InvalidMessage as ex:
				raise InvalidBlock("Invalid block: Message #{} failed validation: {}. In block: {}".format(
					i, str(ex), str(self))
				)

	def __repr__(self):
		return 'Block<hash: {}, prev_hash: {}, messages: {}, time: {}>'.format(
			self.hash, self.prev_hash, len(self.messages), self.timestamp
		)

class SimpleChain:
	def __init__(self):
		self.chain = []

	def add_block(self, block):
		""" Add a block if valid."""
		if len(self.chain) > 0:
			block.prev_hash = self.chain[-1].hash
		block.seal()
		block.validate()
		self.chain.append(block)

	def validate(self):
		""" Validates each block, in order.
			An invalid block invalidates the chain.
		"""
		for i, block in enumerate(self.chain):
			try:
				block.validate()
			except InvalidBlock as exc:
				raise InvalidBlockchain("Invalid blockchain at block number {} caused by: {}".format(i, str(exc)))
		return True

	def __repr__(self):
		return 'SimpleChain<blocks: {}>'.format(len(self.chain))


In [3]:
app = Flask(__name__)

In [4]:
@app.route('/')
def info():
    return \
        json.dumps({
            'server': 'bbserver',
            'version': 0.1,
            'description': 'block-buster blockchain server'
        })


In [5]:
@app.route('/add', methods=['POST'])
def add_block():
    post_data = request.get_json()

    block = Block()
    block.add_message(Message(
        json.dumps(
            post_data
        )
    ))
    chain.add_block((block))

    return \
            json.dumps({
                'status': 'ok',
                'message': 'block added',
                'hash': block.hash
            })

In [6]:
@app.route('/chain')
def show_chain():
    return \
        jsonify([b.__repr__() for b in chain.chain]) # chain.__repr__())


In [7]:
@app.route('/<block_hash>')
def show_block(block_hash):
    return \
        chain.get_block(block_hash).__repr__()


In [8]:
if __name__ == '__main__':
    chain = SimpleChain()
    chain_map = {}

    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [10]:
#http://34.69.195.189:5000/

client.println("/ HTTP/34.69.195.189:5000")

NameError: ignored